# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import pandas
import numpy as np
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.forecasting_parameters import ForecastingParameters
import logging
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automliottime'

experiment=Experiment(ws, experiment_name)

In [ ]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

subscription_id = '6597c678-9946-48a5-a6fe-e27536a47cc0'
resource_group = 'aimazin1'
workspace_name = 'wsi'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='gas')
dataset.to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here

automl_settings = ForecastingParameters(time_column_name='time', 
                   forecast_horizon=50,
                   time_series_id_column_names=["id"],
                   target_lags='auto',
                   target_rolling_window_size=10)

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='forecasting',
                             enable_dnn=True,
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=15,
                             enable_early_stopping=True,
                             training_data=dataset,
                             label_column_name='R7',
                             n_cross_validations=5,
                             enable_ensembling=False,
                             verbosity=logging.INFO,
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:

RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

In [ ]:
#TODO: Save the best model

best_run.register_model(model_path='outputs/auto_model.pkl', model_name='gas_auto',
                        tags={'Training context':'Auto ML'},
                        properties={'normalized_root_mean_squared_error': best_run_metrics['normalized_root_mean_squared_error']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [fitted_model[0]], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

# Load image data
data = [1.25, 1.5]
# Post raw data to scoring URI
res = request.post(url=service[0].scoring_uri, data=data, headers={'R7': np.float()})

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)
service.delete()
